In [1]:
import os

In [2]:
%pwd

'd:\\MLOOPS AI PROJECT\\project-dl-end-to-end-main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MLOOPS AI PROJECT\\project-dl-end-to-end-main'

In [11]:
# mlflow >> to track experiments
# Dagshub is bridge between MLflow and Github

In [ ]:
# https://dagshub.com/kshitiz.dass/project-dl-end-to-end-main.mlflow

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kshitiz.dass/project-dl-end-to-end-main.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kshitiz.dass"
os.environ["MLFLOW_TRACKING_PASSWORD"]="0f6ed27c58b2835ecaea6db1197d2f59acc980f6"

In [9]:
import mlflow
import os

In [10]:
mlflow.set_tracking_uri("https://dagshub.com/kshitiz.dass/project-dl-end-to-end-main.mlflow")
os.environ["MLFLOW_TRACKING_USERNAME"]="kshitiz.dass"
os.environ["MLFLOW_TRACKING_PASSWORD"]="0f6ed27c58b2835ecaea6db1197d2f59acc980f6"

In [12]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [13]:
# entity

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [15]:
# configuration manager

In [18]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/kshitiz.dass/project-dl-end-to-end-main.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [19]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [20]:
# components

In [21]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [22]:
# Pipeline

In [23]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


C:\Users\kshit\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.8906 - loss: 0.2825


2025/04/14 14:50:43 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/14 14:51:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/04/14 14:51:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run dashing-sloth-302 at: https://dagshub.com/kshitiz.dass/project-dl-end-to-end-main.mlflow/#/experiments/0/runs/a424eec75bd8483eb844ff2ff5305d9b
🧪 View experiment at: https://dagshub.com/kshitiz.dass/project-dl-end-to-end-main.mlflow/#/experiments/0
